- Ggf. zu float16 konvertieren checken, ob finaler df mit time slices dann deutlich kleiner und performance testen
- column 'd' in training df löschen?
- paralletl computing einstellen
- use_multiprocessing in keras auf true setzen (model.fit agument)
- Cross validation?
- Ensemble learning?
- brauche ich one-hot encoding für categorical features?

In [1]:
#Import data handling libraries
import pandas as pd
import numpy as np

In [2]:
#Specify directories
#code_env = 'kaggle'
code_env = 'local'


if code_env=='local':
    ###local###
    #get parent folder of current directory
    parent_dir = '/Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction'

    #Directory resources
    res_dir = parent_dir + '/res/'
    src_dir = parent_dir + '/src/'
    prc_dir = src_dir + 'processed_data/' # Processed data directory with pickled dataframes

if code_env=='kaggle':
    ###On Kaggle###
    res_dir = '/kaggle/input/m5-forecasting-accuracy/'
    prc_dir = '/kaggle/input/processed-data-v2/'

In [3]:
# Create variables
BASE     = prc_dir +'df_1.pkl'
CALENDAR = prc_dir +'df_2.pkl'
NUM_ITEMS = 30490 # Number of items per each day

In [4]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense, Masking
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from keras import backend as K

In [5]:
# Read in df_train_conv from pickle file
df_all_data = pd.concat([pd.read_pickle(BASE),
           pd.read_pickle(CALENDAR)], 
           axis=1)

In [6]:
# Define categorical and numerical columns
categorical_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'is_available',
                    'd', 'wday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 
                    'snap_CA', 'snap_TX', 'snap_WI', 'mday', 'week', 'month', 'year']
numerical_cols = ['sell_price']

target_col = 'sales_amount'

# Convert categorical columns to category dtype and encode with cat.codes
for col in categorical_cols:
    df_all_data[col] = df_all_data[col].astype('category').cat.codes

# Normalize numerical columns
scaler = MinMaxScaler()
df_all_data[numerical_cols] = scaler.fit_transform(df_all_data[numerical_cols].astype(np.float32))
df_all_data[target_col] = scaler.fit_transform(df_all_data[[target_col]].astype(np.float32))

In [7]:
# Splitting the data in train, validation and test set; days are now 0 based, so have to shift by 1
TEST_END = 1969 # 1941
VAL_END = TEST_END - 28
TRAIN_END = VAL_END - 28 # 1885 -> Train only until the 28 days before the end of the data

df_train = df_all_data[df_all_data['d'] < TRAIN_END]
df_val = df_all_data[(df_all_data['d'] >= TRAIN_END) & (df_all_data['d'] < VAL_END)]
df_test = df_all_data[df_all_data['d'] >= VAL_END - 7]

del df_all_data

In [8]:
# Custom Generator Function
def lstm_data_generator(df, target, days_per_sequence=1, batch_size=32):
    total_sequences = (len(df) - NUM_ITEMS * days_per_sequence) // NUM_ITEMS # 1878 for train, 21 for val and test
    while True: 
        for i in range(0, total_sequences, batch_size): # 0, 32, 64, ...1878
            batch_sequences = []
            batch_targets = []
            for b in range(batch_size): # 0, 1, 2,... 32
                if i + b < total_sequences: # 0, 0; 0, 1; 0, 2; ...; 0, 32; 32, 0; 32, 1; ...
                    start_idx = (i + b) * NUM_ITEMS
                    end_idx = start_idx + NUM_ITEMS * days_per_sequence
                    batch_sequences.append(df.iloc[start_idx:end_idx, :].drop(target, axis=1).to_numpy()) #drop target column, Only the values in the DataFrame will be returned, the axes labels will be removed.
                    batch_targets.append(df.iloc[end_idx:end_idx + NUM_ITEMS][target].to_numpy())
            yield np.array(batch_sequences), np.array(batch_targets)



# def lstm_data_generator(df, target, days_per_sequence=7, batch_size=32):
#     total_sequences = (len(df) - NUM_ITEMS * (days_per_sequence + 1)) // NUM_ITEMS
#     while True:
#         for i in range(0, total_sequences, batch_size):
#             batch_sequences = []
#             batch_targets = []

#             for b in range(batch_size):
#                 if i + b < total_sequences:
#                     start_idx = (i + b) * NUM_ITEMS
#                     end_idx = start_idx + NUM_ITEMS * days_per_sequence
#                     next_day_idx = end_idx + NUM_ITEMS

#                     # Sequence data with target for past time_step days
#                     sequence_data = df.iloc[start_idx:end_idx, :].copy()
                    
#                     # Adding is_current_day feature to let the model distinguish between the past data and current to predicting day
#                     # create a new column 'is_current_day' and fill it with 0s and set datatype to int8
#                     sequence_data['is_current_day'] = 0
#                     sequence_data['is_current_day'] = sequence_data['is_current_day'].astype(np.int8)

#                     # Data for the current to predicting day without target, because in real life we don't have it
#                     sixth_day_data = df.iloc[end_idx:next_day_idx, :].copy()
#                     #fill column 'sales_amount' with NaNs
#                     sixth_day_data[target] = np.nan
#                     # Give model info that this is the current day
#                     sixth_day_data['is_current_day'] = 1
#                     sixth_day_data['is_current_day'] = sixth_day_data['is_current_day'].astype(np.int8)

#                     # Combine data
#                     sequence_with_sixth_day = pd.concat([sequence_data, sixth_day_data], axis=0)

#                     # Append to batch
#                     batch_sequences.append(sequence_with_sixth_day.to_numpy())

#                     # Target for the 6th day
#                     batch_targets.append(df.iloc[end_idx:next_day_idx, :][target].to_numpy())

#             yield np.array(batch_sequences), np.array(batch_targets)

In [9]:
# Usage
time_steps = 7  # Number of days per sequence
batch_size = 32  # Size of each batch
epochs= 2
train_gen = lstm_data_generator(df_train, target_col, time_steps, batch_size)
val_gen = lstm_data_generator(df_val, target_col, time_steps, batch_size)

In [13]:
# iter through the generator
x, y = next(train_gen)
print(x.shape)
print(y.shape)


(32, 213430, 20)
(32, 30490)


In [11]:
# Custom RMSE loss function
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [17]:
# Assuming you have your LSTM model defined
#input_shape = (time_steps + 1, len(df_train.columns) + 1) input shape for training data from predicting day
input_shape = (time_steps, len(df_train.columns) - 1)
model = Sequential([
    # Masking(mask_value=np.nan, input_shape=input_shape),
    # LSTM(50, activation='tanh'),
    LSTM(50, activation='tanh', input_shape=(input_shape)), #stateful=True return state, return sequences
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', 
              loss='mse', #rmse
              metrics=[RootMeanSquaredError()])
model.summary()


# Add CNN layer
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(time_steps, num_features)))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(LSTM(50, activation='relu'))
# model.add(Dense(1))  # or more layers as needed
# model.compile()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 50)                14200     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 14,251
Trainable params: 14,251
Non-trainable params: 0
_________________________________________________________________


2024-01-11 16:09:28.831524: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-11 16:09:28.832005: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-11 16:09:28.832594: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
# df head with all columns displayed
#pd.set_option('display.max_columns', None)
#df_train.head()

In [ ]:
# Train the model using the generator
# model.fit(x=train_gen, steps_per_epoch=(len(df_train) // (batch_size * NUM_ITEMS)), epochs=3) # x: In case of a generator the target y will be obtained from x; steps_per_epoch: 57mio // (32*30490) = 58

# Fit the model and store history for later evaluation
history = model.fit(
    x=train_gen,
    epochs=epochs,
    steps_per_epoch=(len(df_train) // (batch_size * NUM_ITEMS)),
    validation_data=val_gen,
    validation_steps=(len(df_train) // (batch_size * NUM_ITEMS))
)

In [ ]:
# Save the model to a specified directory
if code_env=='local':
    ###local###
    model.save('/Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction/models/V1_1layer_LSTM.h5')
    
if code_env=='kaggle':
    ###On Kaggle###
    model.save('/kaggle/working/models/V2_1layer_LSTM.h5')

In [ ]:
# Start from here if you want to load the model
from keras.models import load_model

# Load the model from a specified directory
if code_env=='local':
    ###local###
    model = load_model('/Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction/src/models/V1_1layer_LSTM.h5', custom_objects={'rmse': rmse})

if code_env=='kaggle':
    ###On Kaggle###
    model = load_model('/kaggle/working/models/V1_1layer_LSTM.h5', custom_objects={'rmse': rmse})

In [ ]:
import matplotlib.pyplot as plt
try:
    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
except:
    print('No history to plot')

In [27]:
df_test.iloc[0:7*30490, : ].drop(target_col, axis=1).to_numpy()#.shape()

array([[1.437e+03, 3.000e+00, 1.000e+00, ..., 1.900e+01, 4.000e+00,
        5.000e+00],
       [1.438e+03, 3.000e+00, 1.000e+00, ..., 1.900e+01, 4.000e+00,
        5.000e+00],
       [1.439e+03, 3.000e+00, 1.000e+00, ..., 1.900e+01, 4.000e+00,
        5.000e+00],
       ...,
       [1.434e+03, 2.000e+00, 0.000e+00, ..., 1.900e+01, 4.000e+00,
        5.000e+00],
       [1.435e+03, 2.000e+00, 0.000e+00, ..., 1.900e+01, 4.000e+00,
        5.000e+00],
       [1.436e+03, 2.000e+00, 0.000e+00, ..., 1.900e+01, 4.000e+00,
        5.000e+00]], dtype=float32)

In [34]:
# def prepare_forecast_input(df, time_steps, num_items):
#     #df_test starts at 1942-7 which we need take into account
#     # Prepare input data for forecasting
#     forecast_input = []
#     for target_day in range(28):
#         start_idx = target_day * num_items
#         end_idx = start_idx + time_steps * num_items
#         sequence = df.iloc[start_idx:end_idx].drop('sales_amount', axis=1).to_numpy()
#         forecast_input.append(sequence)
#     return np.array(forecast_input)



def prepare_forecast_input(df, target, model, time_steps, num_items):
    forecast_output = []
    for target_day in range(28):
        start_idx = target_day * num_items
        end_idx = start_idx + time_steps * num_items
        sequence = df.iloc[start_idx:end_idx, : ].drop(target, axis=1).to_numpy()
        # forecast_output.append(model.predict(sequence))
        forecast_output.append(model.predict(sequence.reshape(1, sequence.shape[0], sequence.shape[1])))
    return np.array(forecast_output)#.reshape(-1, 1)

forecast_output = prepare_forecast_input(df_test, target_col, model, time_steps, NUM_ITEMS)
#forecasts_original = scaler.inverse_transform(forecast_output)



ValueError: in user code:

    File "/Users/mf/opt/anaconda3/envs/final_bsc/lib/python3.11/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/Users/mf/opt/anaconda3/envs/final_bsc/lib/python3.11/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/mf/opt/anaconda3/envs/final_bsc/lib/python3.11/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/mf/opt/anaconda3/envs/final_bsc/lib/python3.11/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/Users/mf/opt/anaconda3/envs/final_bsc/lib/python3.11/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/mf/opt/anaconda3/envs/fina

In [21]:
# Assuming df_all_data contains all data up to day 1941
# forecast_input = prepare_forecast_input(df_test, time_steps, NUM_ITEMS)

# Generate forecasts
# forecasts = model.predict(forecast_input)
# forecasts_original = scaler.inverse_transform(forecasts)

# forecasts_original now contains the predicted sales amounts for days 1942 to 1969


In [ ]:
# Now, let's define a function to calculate WRMSSE by calculating the RMSSE for each series and then multiplying by the weights and summing them up. 
def calculate_weights(sales_data, last_n_days=28):
    # sales_data: DataFrame with columns ['item_id', 'day', 'sales']
    # Sum sales for each item over the last_n_days
    item_sales = sales_data[sales_data['day'] > sales_data['day'].max() - last_n_days].groupby('item_id')['sales'].sum()
    # Total sales for all items
    total_sales = item_sales.sum()
    # Calculate weights
    weights = item_sales / total_sales
    return weights

def rmsse(y_true, y_pred, h, y_train):
    numerator = np.sum((y_true - y_pred) ** 2) / h
    denominator = np.sum(np.diff(y_train) ** 2) / (len(y_train) - 1) # np.diff to calc the diff for consecutive elements
    return np.sqrt(numerator / denominator)

def wrmsse(y_trues, y_preds, weights, h, y_trains):
    rmsse_values = [rmsse(y_true, y_pred, h, y_train) for y_true, y_pred, y_train in zip(y_trues, y_preds, y_trains)]
    return np.sum(np.array(weights) * np.array(rmsse_values))

In [ ]:
# Evaluate the model on the test set
def evaluate_model_wrmsse(model, df_test, df_train, df_val, batch_size, time_steps, n):
    test_gen = lstm_data_generator(df_test, target_col, time_steps, batch_size)
    steps = max(1, len(df_test) // (batch_size * n))  # Ensure at least 1 step
    y_pred_normalized = model.predict(test_gen, steps=steps)
    y_pred_original = scaler.inverse_transform(y_pred_normalized)
    y_true_normalized = df_test[target_col].values
    y_true_original = scaler.inverse_transform(y_true_normalized)
    
    #First concatenate all elements used for training (df_train and df_val)
    y_train_all_normalized = pd.concat([df_train[target_col], df_val[target_col]], axis=0).values
    y_train_all_original = scaler.inverse_transform(y_train_all_normalized)
    
    # Reshape the predictions and actuals to separate each item's time series
    y_pred_series = [y_pred_original[i::NUM_ITEMS] for i in range(NUM_ITEMS)]
    y_true_series = [y_true_original[i::NUM_ITEMS] for i in range(NUM_ITEMS)]

    # Similarly reshape the training data for RMSSE calculation
    y_train_all_series = [y_train_all_original[i::NUM_ITEMS] for i in range(NUM_ITEMS)]

    # Check - can be deleted later on
    print('len y_pred_series: ' + len(y_pred_series))
    print('len y_true_series: ' + len(y_true_series))
    print('len y_train_all_series: ' + len(y_train_all_series))
    
    # Calculate WRMSSE
    weights = calculate_weights(sales_data)
    wrmsse_score = wrmsse(y_trues=y_true_series, y_preds=y_pred_series, weights=weights, h=28, y_trains=y_train_all_series)

    print("Test WRMSSE: ", wrmsse_score)
    
    
    
    
    
    
    
    
    
    # Calculate wrmsse score
    wrmsse_score = wrmsse(
        y_trues=y_true_original,
        y_preds=y_pred_original,
        weights=calculate_weights(sales_data),
        h=28, # forecast horizon
        y_train=y_train_all_original
    )
    print("Test WRMSSE: ", wrmsse_score)

In [ ]:
# Call the evaluate function
evaluate_model_wrmsse(model, df_test, df_train, df_val, batch_size, time_steps, VAL_END)